In [7]:
import os
from tqdm.notebook  import tqdm
from collections import defaultdict
import pandas as pd

In [8]:
result_folder = r'D:\flann-orb-distances-330sm'

In [9]:
txt_files = os.listdir(result_folder)
txt_files = [os.path.join(os.path.abspath(result_folder), f) for f in txt_files]
txt_files[:2]

['D:\\flann-orb-distances-330sm\\149781103607370_added-7706.txt',
 'D:\\flann-orb-distances-330sm\\2018_12-10375.txt']

In [10]:
if not os.path.exists("parquets-330"):
    os.makedirs("parquets-330")

In [11]:
rows = defaultdict(dict)
for i, file in enumerate(tqdm(txt_files, total=len(txt_files), unit=' file')):
    meme_id = os.path.basename(os.path.splitext(file)[0])
    distances = {}
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip().split('\t')
            dists = line[1].split(',')
            dists = [int(float(d)) for d in dists]
            dists.sort()
            distances[line[0]] = dists[:50]
    rows[meme_id] = distances
    if i % 100 == 0 and i > 0:
        # create a pandas pivoted dataframe where the values are the list of distances
        df = pd.DataFrame(rows).T
        df.to_parquet(f'./parquets-330/imgflip-test-distances-part-{i}.parquet', compression='gzip')
        rows = defaultdict(dict)

  0%|          | 0/330 [00:00<?, ? file/s]

In [12]:
df = pd.DataFrame(rows).T
df.to_parquet(f'./parquets-330/imgflip-test-distances-part-last.parquet', compression='gzip')